# 電算倶楽部 2022/12/10 WisperTest

## See also
https://dev.classmethod.jp/articles/whisper-how-to/

In [9]:
!python3.9.exe -m pip install pip --upgrade
!python3.9.exe -m pip install git+https://github.com/openai/whisper.git
!python3.9.exe -m pip install torch===1.13.1 torchvision===0.14.1 -f　https://download.pytorch.org/whl/torch_stable.html

# %python3.11 -m pip install whisper


  Cloning https://github.com/openai/whisper.git to c:\users\shimp\appdata\local\temp\pip-req-build-8ec3wel5
  Resolved https://github.com/openai/whisper.git to commit 7858aa9c08d98f75575035ecd6481f462d66ca27
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\shimp\AppData\Local\Temp\pip-req-build-8ec3wel5'


Looking in links: 　https://download.pytorch.org/whl/torch_stable.html
  Using cached torch-1.13.1-cp39-cp39-win_amd64.whl (162.5 MB)
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     -------------- ------------------------- 0.4/1.1 MB 12.9 MB/s eta 0:00:01
     ------------------------------- -------- 0.8/1.1 MB 10.8 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] アクセスが拒否されました。: 'C:\\Users\\shimp\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python39\\site-packages\\~orch\\lib\\asmjit.dll'
Check the permissions.



In [10]:
import whisper
from whisper.transcribe import format_timestamp
from typing import Tuple, List



In [20]:

model = whisper.load_model("large", device='cpu')
_ = model.half()

# model.cuda()

AssertionError: Torch not compiled with CUDA enabled

In [21]:
def jimaku_srt(start_end_texts: List[Tuple[float, float, str]]) -> str:
  res: List[str] = []
  for i, (start,end,text) in enumerate(start_end_texts):
    start_str = format_timestamp(start, always_include_hours=True, decimal_marker=',')
    end_str = format_timestamp(end, always_include_hours=True, decimal_marker=',')
    res.append(f"{i+1}")
    res.append(f"{start_str} --> {end_str}")
    res.append(f"{text}")
    res.append("")
  return '\n'.join(res)


print(jimaku_srt([(1, 2, "he")]))


# sample
# =========
# 1
# 00:00:05,000 --> 00:00:10,000
# これは
# 
# 2
# 00:00:10,000 --> 00:00:15,000
# ｋ本的に
# （きほんてきに）
# 
# 3
# 00:00:15,000 --> 00:00:20,000
# テストです。
# 
# 4
# 00:00:20,000 --> 00:00:25,000
# こんにちわ

1
00:00:01,000 --> 00:00:02,000
he



In [22]:
def jimaku_webvtt(title: str, start_end_texts: List[Tuple[float, float, str]]) -> str:
  res: List[str] = []
  res.append(f"WEBVTT - {title}")
  for i, (start,end,text) in enumerate(start_end_texts):
    start_str = format_timestamp(start, always_include_hours=True, decimal_marker='.')
    end_str = format_timestamp(end, always_include_hours=True, decimal_marker='.')
    res.append(f"{i+1}")
    res.append(f"{start_str} --> {end_str}")
    res.append(f"{text}")
    res.append("")
  return '\n'.join(res)


# sample
# =======
# WEBVTT - 好きな映画の翻訳
# 
# NOTE
# 何人かの友人が彼らの両親と一緒にそれを見ることができるように、
# この翻訳は Kyle によってされました。
# 
# 1
# 00:02:15.000 --> 00:02:20.000
# - Ta en kopp varmt te.
# - Det är inte varmt.
# 
# 2
# 00:02:20.000 --> 00:02:25.000
# - Har en kopp te.
# - Det smakar som te.
# 
# NOTE この最後の行はうまく翻訳されていないかもしれません。
# 
# 3
# 00:02:25.000 --> 00:02:30.000
# - Ta en kopp

In [23]:
def kashi_lrc(start_texts: List[Tuple[float, str]]) -> str:
  res: List[str] = []
  for (start,text) in start_texts:
    start_str = format_timestamp(start, always_include_hours=True, decimal_marker='.')
    res.append(f"[{start_str}]{text}")
  return '\n'.join(res)
# Sapmle
# ========
# [00:05.00]ありがとう
# [00:13.00] 
# [00:19.00]はじめまして

print(kashi_lrc([(1, "he")]))

[00:00:01.000]he


In [24]:
import os
import json

def collect_audio(dir_pathes: str) -> List[str]:
  exts = [".wav", ".mp3", ".ogg", ".mp4", ".mkv", ".avi", ".acc", ".m4a", ".opus", ".wmv", ".wma", ".mp9", ".webm"]
  results = []
  for dir_path in dir_pathes:
    for root, _, files in os.walk(dir_path):
      for file in files:
        ext = os.path.splitext(file)[1]
        if ext in exts:
          results.append(os.path.join(root,file))
  return results


dir_pathes = [
  r"E:\文字起こし待ち",
  r"C:\Users\shimp\Videos\game",
  r"E:\onlinestrage\import_Evernote\Whisper",
  r"E:\onlinestrage\import_Evernote\アップロード待機\20221119_hat202",
  r"H:\LoiLo Game\スマホ動画無圧縮\20190719",
  # r"E:\onlinestrage\import_Evernote\アップロード待機",
]
audio_pathes = collect_audio(dir_pathes)
print(audio_pathes)


['E:\\文字起こし待ち\\ダメ人間会_2023-01-14 23-11-33.mkv', 'E:\\文字起こし待ち\\ダメ人間会_2023-01-20 23-11-30.mkv', 'E:\\文字起こし待ち\\ダメ人間会_2023-01-21 23-23-21.mkv', 'E:\\文字起こし待ち\\ダメ人間会_2023-01-27 22-41-25.mkv', 'E:\\文字起こし待ち\\ダメ人間会_2023-02-03 22-09-15.mkv', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20180920_無修正 ライブチャット 素人 美少女 オナニー アイドル 可愛い 配信 チャ バイブ.mp4', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20181107_Mat438_1.mp4', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20190215_15232137.mp4', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20190331_rok859_rok (5).mp4', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20200317_cha817_ch03.mp4', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20200322_cha817_ch04_ch04 (4).mp4', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20201025_ari281_ari281_ar08_600609262.564195.mp4', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20220412_上戸彩\u3000胸揉まれる.wmv', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20220723_nin178_ni02.mp4', 'E:\\onlinestrage\\import_Evernote\\Whisper\\20180428_liv

In [25]:

def get_language(audio_path: str) -> str:
  if 'LianaGood' in audio_path:
    return 'en'
  else:
    return 'ja'

results = []
for i, audio_path in enumerate(audio_pathes):
  print(f"[{i+1}/{len(audio_pathes)}] {audio_path}")
  
  filebasename = os.path.splitext(os.path.basename(audio_path))[0]
  dirpath = os.path.dirname(audio_path)
  all_filename = filebasename + '_whisper.json'
  all_filepath = os.path.join(dirpath, all_filename)
  if os.path.exists(all_filepath):
    continue
  text_filename = filebasename + '_whisper.txt'
  text_filepath = os.path.join(dirpath, text_filename)
  srt_filename = filebasename + '_whisper.srt'
  srt_filepath = os.path.join(dirpath, srt_filename)
  webvtt_filename = filebasename + '_whisper.vtt'
  webvtt_filepath = os.path.join(dirpath, webvtt_filename)
  lrc_filename = filebasename + '.lrc'
  lrc_filepath = os.path.join(dirpath, lrc_filename)

  lang = get_language(audio_path)

  result = model.transcribe(audio_path, verbose=False, fp16=False, language=lang)
  results.append((audio_path, result))
  # print(result["text"])

  with open(all_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(json.dumps(result, ensure_ascii=False, indent=2))
  with open(text_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(result["text"])
  start_end_texts = [(x["start"], x["end"], x["text"]) for x in result["segments"]]
  with open(srt_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(jimaku_srt(start_end_texts))
  with open(webvtt_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(jimaku_webvtt(filebasename, start_end_texts))
  with open(lrc_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(kashi_lrc([(x[0], x[2]) for x in start_end_texts]))


[1/35] E:\文字起こし待ち\ダメ人間会_2023-01-14 23-11-33.mkv


  0%|          | 0/1148793 [00:00<?, ?frames/s]


RuntimeError: expected scalar type Float but found Half

In [ ]:
for audio_path, result in results:
  print(audio_path)
  filebasename = os.path.splitext(os.path.basename(audio_path))[0]
  dirpath = os.path.dirname(audio_path)
  all_filename = filebasename + '_whisper.json'
  all_filepath = os.path.join(dirpath, all_filename)
  text_filename = filebasename + '_whisper.txt'
  text_filepath = os.path.join(dirpath, text_filename)
  srt_filename = filebasename + '_whisper.srt'
  srt_filepath = os.path.join(dirpath, srt_filename)
  webvtt_filename = filebasename + '_whisper.vtt'
  webvtt_filepath = os.path.join(dirpath, webvtt_filename)
  lrc_filename = filebasename + '.lrc'
  lrc_filepath = os.path.join(dirpath, lrc_filename)

  with open(all_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(json.dumps(result, ensure_ascii=False, indent=2))
  with open(text_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(result["text"])
  start_end_texts = [(x["start"], x["end"], x["text"]) for x in result["segments"]]
  with open(srt_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(jimaku_srt(start_end_texts))
  with open(webvtt_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(jimaku_webvtt(filebasename, start_end_texts))
  with open(lrc_filepath, 'w', encoding='utf-8') as fp:
    fp.writelines(kashi_lrc([(x[0], x[2]) for x in start_end_texts]))

E:\onlinestrage\import_Evernote\アップロード待機\20221119_hat202\hat2023.mp4
H:\LoiLo Game\スマホ動画無圧縮\20190719\QVR_2019_07_19_10_01_43.mp9


# 新しいセクション

In [ ]:
result = model.transcribe("audio_sample.wav", verbose=True, language="ja", task="translate")
print(result["text"])
